In [1]:
!pip install rasterio
!pip install ipygee
!pip install tslearn
!pip install earthengine-api
!pip install geopandas

In [9]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWgu8F6GkK8r8i3yIDlg4jelPJbLBrNOdE1PyJS2dsaTR3H-Dz_OVgU

Successfully saved authorization token.


In [10]:
import folium
from folium import plugins
from IPython.display import Image
import geopandas as gpd
import json
print(folium.__version__)
from ipygee import*
import math
import pandas as pd
from tslearn.clustering import TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset

0.12.1


In [11]:
AOI =  ee.Geometry.Polygon(
[[[
              32.23388671875,
              -17.371610024104744
            ],
            [
              37.08984375,
              -17.371610024104744
            ],
            [
              37.08984375,
              -9.123792057073972
            ],
            [
              32.23388671875,
              -9.123792057073972
            ],
            [
              32.23388671875,
              -17.371610024104744
            ]]])
points = ee.FeatureCollection.randomPoints(AOI,100)

In [12]:
months = ee.List.sequence(1,12)
years = ee.List.sequence(2018, 2019)

In [18]:
MD_NDVI = ee.ImageCollection('MODIS/MOD09GA_006_NDVI').filterDate('2018-1-1','2019-12-31').filterBounds(AOI).select('NDVI')
modis_ndvi = MD_NDVI.median().clip(AOI)
mean_ndvi = MD_NDVI.mean().clip(AOI)

In [19]:
vis_params = {'min': 0, 'max': 1, 'palette': 
['blue', 'yellow', 'green', 'darkgreen','black']}
basemaps = {
'Google Maps': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Maps',
overlay = True,
control = True
),
'Google Satellite': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Satellite',
overlay = True,
control = True
)}
def add_ee_layer(self, ee_object, vis_params, name):
 try:
  if isinstance(ee_object, ee.image.Image):
   map_id_dict = ee.Image(ee_object).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object, ee.imagecollection.ImageCollection):
   ee_object_new = ee_object.mosaic()
   map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name, 
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object, ee.geometry.Geometry):
   folium.GeoJson(
   data = ee_object.getInfo(),
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object,ee.featurecollection.FeatureCollection):
   ee_object_new = ee.Image().paint(ee_object, 0, 2)
   map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

 except:
  print("Could not display {}".format(name))
folium.Map.add_ee_layer = add_ee_layer
my_map = folium.Map(location=[-13.2512161, 34.3015278],
                    zoom_start=10)
basemaps['Google Maps'].add_to(my_map)
my_map.add_ee_layer(modis_ndvi, vis_params, 'NDVI')
my_map.add_ee_layer(points.geometry(), {}, 'Points')
my_map.add_child(folium.LayerControl())
display(my_map)

In [20]:
def monthly(collection):
 img_coll = ee.ImageCollection([])
 for y in years.getInfo():
  for m in months.getInfo():
   filtered = collection.filter(ee.Filter.calendarRange(y,y,'year')).filter(ee.Filter.calendarRange(m, m, 'month'))
   filtered = filtered.median()
   img_coll = img_coll.merge(filtered.set('year', y).set('month', m).set('system:time_start',ee.Date.fromYMD(y, m, 1).getInfo()['value']))
 return img_coll
Monthly_MD = monthly(MD_NDVI)

In [21]:
Point_1 = ee.Geometry.Point([-13.2512161, 34.3015278])
MD_ndvi = chart.Image.series(**{'imageCollection': Monthly_MD,
'region': Point_1,
'reducer': ee.Reducer.mean(),
'scale': 250,
'xProperty': 'system:time_start'})
MD_ndvi.renderWidget(width='80%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…